In [1]:
%load_ext autoreload
%autoreload 2
%run -n main.py

# sources

In [27]:
# %run -n main.py
# for name in [SENT, TOKEN]:
#     path = join_path(DATA_DIR, name, SOURCE)
#     !mkdir -p {path}

In [5]:
# %run -n main.py
# for type in [TOKEN, SENT]:
#     log(type)
#     for name in [CORPORA, SYNTAG, GICRYA, RNC]:
#         paths = (
#             join_path(CORUS_DATA_DIR, _)
#             for _ in CORUS_FILES[name]
#         )
#         records = (
#             record
#             for path in paths
#             for record in SOURCES[name](path)
#         )
#         records = log_progress(records, desc=name)
#         partitions = PARSES[type][name](records)
#         lines = format_partitions(partitions)
#         path = join_path(DATA_DIR, type, SOURCE, name + JL + GZ)
#         dump_gz_lines(lines, path)

# segmenters

In [5]:
# from type in [TOKEN, SENT]:
#     for name in SEGS[type]:
#         path = join_path(DATA_DIR, type, name)
#         !mkdir -p {path}

In [ ]:
# !pip install rusenttokenize
# !pip install segtok
# !pip install mosestokenizer
# !pip install razdel

# !pip install spacy
# !pip install https://github.com/aatimofeev/spacy_russian_tokenizer/archive/master.zip#egg=python-simhash
# !pip install pymystem3

# !pip install nltk
# nltk.download('punkt')
# !wget https://raw.githubusercontent.com/mhq/train_punkt/master/russian.pickle -O ~/nltk_data/tokenizers/punkt/PY3/russian.pickle

# Texterra
# Можно ещё сравнивать с https://texterra.ispras.ru/products, но
# 1. она медленно работает, как минимум затраты на http
# 2. иногда кидает ошибку (возможно дело в английских предложениях)
# 3. качество немного выше segtok

# Polyglot
# реализует http://www.unicode.org/reports/tr29/

# Сорян, не смог установить. Дикие траблы с ICU
# brew install icu4c
# export ICU_VERSION=62.1
# export BASE=/usr/local/Cellar/icu4c/
# export PATH=$PATH:$BASE/$ICU_VERSION/bin
# export PYICU_INCLUDES=$BASE/$ICU_VERSION/include
# export PYICU_LFLAGS=-L$BASE/$ICU_VERSION/lib
# pip install pyicu polyglot

# Вроде установилось но 
# > from polyglot.text import Text
# > Text('...')
# Symbol not found: __ZNK6icu_6214Transliterator12getTargetSetERNS_10UnicodeSetE

In [4]:
# for type in [SENT, TOKEN]:
#     log(type)
#     for seg_name in SEGS[type]:
#         log(seg_name)
#         seg = SEGS[type][seg_name]
#         if is_class(seg):
#             seg = seg()
#         seg = Timing(seg)

#         for source_name in SOURCES:
#             path = join_path(DATA_DIR, type, SOURCE, source_name + JL + GZ)
#             lines = load_gz_lines(path)
#             records = parse_partitions(lines)
#             records = head(records, 10000)
#             records = log_progress(records, desc=source_name)

#             records = (
#                 Partition.from_substrings(seg(_.text))
#                 for _ in records
#             )
#             path = join_path(DATA_DIR, type, seg_name, source_name + JL + GZ)
#             lines = format_partitions(records)
#             dump_gz_lines(lines, path)

#             path = join_path(DATA_DIR, STATS + JL)
#             record = [[type, seg_name, source_name], seg.time]
#             lines = format_jl([record])
#             append_lines(lines, path)

#             seg.reset()

# score

In [2]:
scores = {}
keys = [
    (type, seg, source)
    for type in [TOKEN, SENT]
    for seg in SEGS[type]
    for source in SOURCES
]
for type, seg, source in log_progress(keys):
    path = join_path(DATA_DIR, type, SOURCE, source + JL + GZ)
    lines = load_gz_lines(path)
    targets = parse_partitions(lines)

    path = join_path(DATA_DIR, type, seg, source + JL + GZ)
    lines = load_gz_lines(path)
    preds = parse_partitions(lines)

    score = score_partitions(preds, targets)
    scores[type, seg, source] = score

# report

In [4]:
path = join_path(DATA_DIR, STATS + JL)
lines = load_lines(path)
items = parse_jl(lines)
times = {
    tuple(key): time
    for key, time in items
}

In [25]:
table = report_table(scores, times, SOURCES, SEGS, TOKEN)
html = format_report(table, github=True)
HTML(html)

In [26]:
table = report_table(scores, times, SOURCES, SEGS, SENT)
html = format_report(table, github=True)
HTML(html)